# TensorFlow 2 + Keras Model -> CoreML

This notebook loads weights trained in a TensorFlow 2 DCGAN model to a TensorFlow 1 + Keras model, and converts that to CoreML.

## Installation / Environment

You'll need a working Python 3 environment with **TensorFlow 1 and Keras** installed (and Jupyter, of course). For stability, I've chosen:
 - keras==2.2.4
 - tensorflow-gpu==1.13.1
 - coremltools==3.4

## coremltools and TensorFlow 2
[coremltools 4.0b1](https://github.com/apple/coremltools/releases) has improved TF 2 conversion support, but I've had issues with it. Could be user error, but redefining the model with TF 1 is easy enough.

For more about coremltools 4.0, check out the [documentation here](https://coremltools.readme.io/docs).

That said I'm optimistic about streamlined TF 2 conversion in the near future! Hopefully this entire notebook becomes uncessary soon.

In [ ]:
import keras
from keras import layers
from keras.initializers import TruncatedNormal

import os

import coremltools as ct
from coremltools.proto import FeatureTypes_pb2 as ft 

# Model Redefinition
This model is an exact redefinition an existing TensorFlow 2 model. It's the same architecture, but uses `keras` instead of `tensorflow.keras`.

Input is a 128x1 vector, Output is a 128x128x3 image.

### READ THE MODEL SUMMARY!
The model summary should match the TF 2 model summary *exactly*. If not, the weights won't load.

In [ ]:
latent_dim = 128

def build_model():
    model = keras.Sequential()
    model.add(layers.Dense(256 * 16 * 16,
                    use_bias=False,
                    input_dim=latent_dim))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Reshape((16, 16, 256)))
    model.add(layers.UpSampling2D())

    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(1, 1),
                                     use_bias=False,
                                     padding="same",
                                     kernel_initializer=TruncatedNormal(stddev=0.02),
                                     data_format='channels_last'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.2))

    model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2),
                                     use_bias=False,
                                     padding="same",
                                     kernel_initializer=TruncatedNormal(stddev=0.02),
                                     data_format='channels_last'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.2))
    
    model.add(layers.Conv2DTranspose(32, (5, 5), strides=(2, 2),
                                     use_bias=False,
                                     padding="same",
                                     kernel_initializer=TruncatedNormal(stddev=0.02),
                                     data_format='channels_last'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.2))

    model.add(layers.Conv2D(3, (5, 5), strides=(1, 1), activation='tanh', padding='same'))
    return model

model = build_model()
model.summary()

# Load model weights

In this example I'm loading the generator from my DCGAN notebook.

In [ ]:
model_name = 'MODEL_FOLDER'
model_path = os.path.join('models', model_name)
generator_weights_path = os.path.join(model_path, 'generator_weights.hdf5')
mlmodel_path = os.path.join(model_path, 'Model.mlmodel')
model.load_weights(generator_weights_path)

# CoreML Conversion

Use these cells to convert the model to CoreML using coremltools.

For ease-of-use with CoreML, it's important to convert the output type from multiArrayType to imageType.

Alternatively, you can convert MLMultiArray to an Image manually at inference time, but I wouldn't recommend it. There's much more on the subject by [Matthijs Hollemans](https://machinethink.net/blog/coreml-image-mlmultiarray/)

In [ ]:
mlmodel = ct.converters.keras.convert(
    model,
    input_names=["noise_in"]
)
spec = mlmodel.get_spec()
print(spec.description)

## Postprocessing

The model currently outputs a 128x128x3 RGB array with values in [-1, 1]. Let's add some postprocessing to scale that output to [0, 255].

For more on CoreML model i/o, see [How to convert images to MLMultiArray](https://machinethink.net/blog/coreml-image-mlmultiarray/) and [How to Get Core ML Proudce Images as Output](https://cutecoder.org/featured/core-ml-image-output/).

In [ ]:
# Store a reference to the last layer
spec_layers = getattr(spec, spec.WhichOneof("Type")).layers
last_layer = spec_layers[-1]

# Create a new activation layer that scales output to [0, 255]
new_layer = spec_layers.add()
new_layer.name = 'image_out'
new_layer.activation.linear.alpha = 127.5
new_layer.activation.linear.beta = 127.5
new_layer.output.append('image_out')

# Point the last layer's output to the new layer
new_layer.input.append(last_layer.output[0])

# Find the original model's output description
output_description = next(x for x in spec.description.output if x.name==last_layer.output[0])

# Update the model output to use the new layer
output = spec.description.output[0]
output.name = new_layer.name
output.type.imageType.colorSpace = ft.ImageFeatureType.RGB
output.type.imageType.width = 128
output.type.imageType.height = 128

print(spec.description)

In [ ]:
updated_model = ct.models.MLModel(spec)
updated_model.save(mlmodel_path)
print("Saved mlmodel file to {}".format(mlmodel_path))

# Using the model (in Swift)

To use the model in your iOS project, start by adding it to your project.

XCode will automatically generate a class for your model based on the mlmodel file. Click on the mlmodel file to see information about the generated class name. While you're there, confirm the input and output types of your model.

**You may need to build once to silence XCode warnings**.

The gist of things is that we'll be using MLMultiArray to represent latent vectors, with the goal of creating UIImage output that we can work with easily.

To start, we can use [GKRandomDistribution](https://developer.apple.com/documentation/gameplaykit/gkgaussiandistribution) to generate normally distributed random vectors:
```swift
// Generate a noise vector (MLMultiArray) with n dimensions
func createNoise(_ n: Int = 128, distribution: GKGaussianDistribution) -> MLMultiArray {
    guard let mlArray = try? MLMultiArray(shape: [n as NSNumber], dataType: .float32) else {
        fatalError("Unexpected runtime error. MLMultiArray")
    }
    for idx in (0..<n) {
        mlArray[idx] = distribution.nextUniform() as NSNumber
    }
    return mlArray
}
```

If you need random numbers from a different mean / standard deviation, check out [this answer on Stackoverflow](https://stackoverflow.com/a/49471411).

The autogenerated CoreML model class will generate CVPixelBuffer output. There are a [few ways](https://machinethink.net/blog/coreml-image-mlmultiarray/) to get that into a UIImage. I've been using a small UIImage Extension:
```swift
import VideoToolbox

extension UIImage {
    public convenience init?(pixelBuffer: CVPixelBuffer) {
        var cgImage: CGImage?
        VTCreateCGImageFromCVPixelBuffer(pixelBuffer, options: nil, imageOut: &cgImage)

        if let cgImage = cgImage {
            self.init(cgImage: cgImage)
        } else {
            return nil
        }
    }
}
```

Finally, we can instantiate the model and noise distribution, call prediction() with the a noise vector, and turn the result into a UIImage:
```swift
import CoreML
import GameKit

var model = flowers_v1() // autogenerated model class
let random = GKRandomSource()
let noiseDistribution = GKGaussianDistribution(randomSource: random, mean: 0, deviation: 1)

// Create a noise vector and run a prediction
let ganInput = createNoise(128, distribution: noiseDistribution)
guard let transformed = try? model.prediction(noise_in: ganInput) else {
    print("Failed to predict")
}
let ganImage = UIImage(pixelBuffer: transformed.image_out)!
```